__Import Libraries__

In [3167]:
from ultralytics import YOLO
from pathlib import Path
from collections import defaultdict
from hungarian import * 
from pprint import pprint 
from PIL import Image, ImageDraw
import os
import cv2
import torch
import json
import math
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%run Radar_Clustering_CustomDBScan.py

__Path Definition__

In [3168]:
'''
source_dir = Path(r"C:\Dk\Projects\Team Project\Dataset\INFRA-3DRC-Dataset")

image_list = []
pcd_list = []
calibration_list = []
scenes = [f.path for f in os.scandir(source_dir) if f.is_dir()]
for scene in scenes:
    # Get the path to the 'camera_01__data' directory
    image_dir = Path(scene) / 'camera_01' / 'camera_01__data'
    images = sorted(list(image_dir.rglob('*.png')))
    image_list.append(images)

    pcd_dir = Path(scene) / 'radar_01' / 'radar_01__data'
    pcds = sorted(list(pcd_dir.rglob('*.pcd')))
    pcd_list.append(pcds)

    calibration_path = Path(scene)
    calibration_dir = list(calibration_path.rglob('calibration.json'))
    calibration_list.extend(calibration_dir)

yolo_model = YOLO(r"C:\Dk\Projects\Team Project\YOLO detection\Models\Harshit_Large\large_300 epoch_batch 4_augmented\train32\weights\best.pt")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
'''

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\samco\AppData\Local\Temp\ipykernel_30096\2588889635.py:1: SyntaxWarning: invalid escape sequence '\D'
  '''


'\nsource_dir = Path(r"C:\\Dk\\Projects\\Team Project\\Dataset\\INFRA-3DRC-Dataset")\n\nimage_list = []\npcd_list = []\ncalibration_list = []\nscenes = [f.path for f in os.scandir(source_dir) if f.is_dir()]\nfor scene in scenes:\n    # Get the path to the \'camera_01__data\' directory\n    image_dir = Path(scene) / \'camera_01\' / \'camera_01__data\'\n    images = sorted(list(image_dir.rglob(\'*.png\')))\n    image_list.append(images)\n\n    pcd_dir = Path(scene) / \'radar_01\' / \'radar_01__data\'\n    pcds = sorted(list(pcd_dir.rglob(\'*.pcd\')))\n    pcd_list.append(pcds)\n\n    calibration_path = Path(scene)\n    calibration_dir = list(calibration_path.rglob(\'calibration.json\'))\n    calibration_list.extend(calibration_dir)\n\nyolo_model = YOLO(r"C:\\Dk\\Projects\\Team Project\\YOLO detection\\Models\\Harshit_Large\\large_300 epoch_batch 4_augmented\train32\\weights\x08est.pt")\ndevice = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\n'

In [3169]:

path_to_images = Path(r'C:\Dk\Projects\Team Project\Dataset\INFRA-3DRC-Dataset\INFRA-3DRC_scene-15\camera_01\camera_01__data')
path_to_pcd = Path(r'C:\Dk\Projects\Team Project\Dataset\INFRA-3DRC-Dataset\INFRA-3DRC_scene-15\radar_01\radar_01__data')

scene_image = sorted(list(image for image in path_to_images.iterdir()))
scene_pcd = sorted(list(image for image in path_to_pcd.iterdir()))

# scene_image = scene_image[30:31]
# scene_pcd = scene_pcd[30:31]

yolo_model = YOLO(r"C:\Dk\Projects\Team Project\YOLO detection\Models\Harshit_Large\large_300 epoch_batch 4_augmented\train32\weights\best.pt")

calibration_file = Path(r"C:\Dk\Projects\Team Project\Dataset\INFRA-3DRC-Dataset\INFRA-3DRC_scene-15\calibration.json")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


__Function to Process YOLO prediction results__

In [3170]:
def class_box_generator_for_pred(prediction_results):
    for result in prediction_results:
        cls = result.boxes.cls.cpu().numpy()
        conf = result.boxes.conf.cpu().numpy()
        detection = result.boxes.xyxy.cpu().numpy()

        list_of_pred_boxes = np.column_stack((cls, detection, conf))
    
    return list_of_pred_boxes

__Import Calibration Matrix__

In [3171]:
def get_sensor_calibration_dict(calibration_file):
    sensor_calibration_dict = {
        "camera_intrinsics": [],
        "camera_distcoeffs": [],
        "radar_to_camera": [],
        "radar_to_lidar": [],
        "lidar_to_ground": [],
        "camera_to_ground": []
    }

    with open(calibration_file, 'r') as f:
        data = json.load(f)

    for item in data['calibration']:
        if item['calibration'] == 'camera_01':
            sensor_calibration_dict['camera_intrinsics'] = item['k']
            sensor_calibration_dict['camera_distcoeffs'] = item['D']
        elif item['calibration'] == 'radar_01_to_camera_01':
            sensor_calibration_dict['radar_to_camera'] = item['T']
        elif item['calibration'] == 'radar_01_to_lidar_01':
            sensor_calibration_dict['radar_to_lidar'] = item['T']
        elif item['calibration'] == 'lidar_01_to_ground':
            sensor_calibration_dict['lidar_to_ground'] = item['T']
        elif item['calibration'] == 'camera_01_to_ground_homography':
            sensor_calibration_dict['camera_to_ground'] = item['T']

    return sensor_calibration_dict

__Calibration: Radar to Ground Plane__

In [3172]:
def radar_to_ground_transfomer(points_array, T, K):

    n_p_array = np.array(points_array).reshape(1,-1)
    tranposed_array = np.transpose(n_p_array)
   
    row_of_ones = np.ones((1, 1))           #1x1
    stacked_matrix = np.vstack((tranposed_array, row_of_ones))  
  
    radar_to_lidar_matrix = np.matmul(T, stacked_matrix)             #3x1

    new_stacked_matrix = np.vstack((radar_to_lidar_matrix, row_of_ones))             #4x1
    in_ground_data = np.matmul(K, new_stacked_matrix)


    in_ground = np.transpose(in_ground_data)

    return in_ground[0]


__Radar dict: on Ground__

In [3173]:
def radar_to_ground(radar_dict, sensor_calibration_dict):
    
    T = sensor_calibration_dict['radar_to_lidar']
    K = sensor_calibration_dict['lidar_to_ground']

    in_radar = radar_dict
    in_ground = {'clusters': [], 'noise': []}
    for key, value in in_radar.items():
        if key == 'clusters':
            for point in value:
                if point:
                    updated_centroid = radar_to_ground_transfomer(point[0], T, K)
                    updated_lowest_point = radar_to_ground_transfomer(point[1], T, K)
                    updated_velocity = point[2]
                    updated_point = [list(updated_centroid), list(updated_lowest_point), list(updated_velocity)]

                    if key in in_ground:
                        in_ground[key].append(updated_point)
                    else:
                        print('no key exist')
        else:
            for point in value:
                if point:
                    updated_centroid = radar_to_ground_transfomer(point[0], T, K)
                    updated_velocity = point[1]
                    updated_point = [list(updated_centroid), list(updated_velocity)]

                    if key in in_ground:
                        in_ground[key].append(updated_point)
                    else:
                        print('no key exist')
                    
    return in_ground

__Calibration: Radar to Image Plane__

In [3174]:
def radar_to_camera_transformer(radar_point, T, k):
   
    n_p_array = np.array(radar_point).reshape(1,-1)
    transpose_RPA = np.transpose(n_p_array)

    new_array = np.vstack([transpose_RPA, np.ones((1, 1))])             
    product_1 = np.matmul(np.array(k), np.array(T))

    product_array = np.matmul(product_1, new_array)                      #[su, sv, s] but along column

    final_array = product_array / product_array [2]                      #[u, v, 1], along column

    u_v = np.delete(final_array, 2, axis = 0)                            #[u, v], along column      
    final_u_v = np.transpose(u_v)

    return final_u_v[0]

__Radar Dict: on Image__

In [3175]:
def radar_to_camera(radar_output, sensor_calibration_dict):
    T =  sensor_calibration_dict['radar_to_camera']
    K = sensor_calibration_dict['camera_intrinsics']
    
    in_radar = radar_output
    in_camera = {'clusters': [], 'noise': []}
    for key, value in in_radar.items():
        if key == 'clusters':
            for point in value:
                if point:
                    updated_centroid = radar_to_camera_transformer(point[0], T, K)
                    updated_lowest_point = radar_to_camera_transformer(point[1], T, K)
                    updated_velocity = point[2]
                    updated_point = [list(updated_centroid), list(updated_lowest_point), list(updated_velocity)]

                    if key in in_camera:
                        in_camera[key].append(updated_point)
                    else:
                        print('no key exist')

        if key == 'noise':
            for point in value:
                if point:
                    updated_centroid = radar_to_camera_transformer(point[0], T, K)
                    updated_velocity = point[1]
                    updated_point = [list(updated_centroid), list(updated_velocity)]

                    if key in in_camera:
                        in_camera[key].append(updated_point)
                    else:
                        print('no key exist')

    return in_camera

__Homography: Image to Ground__

In [3176]:
'''
def homography2(list_of_pred_boxes):
    ground_coordinate_list = []
    for result in list_of_pred_boxes:
        bbox = list(result[1:5])
    
        # x1y1 = np.array(bbox[:2]).reshape(1, -1)
        # x2y2 = np.array(bbox[2:]).reshape(1, -1)
        bottom_center_point = np.array(list(((bbox[2] + bbox[0]) / 2, bbox[3]))).reshape(1, -1) 

        # image_coordinates = np.concatenate((x1y1,x2y2,bottom_center_point), axis=0)
        # image_coordinates = np.concatenate((bottom_center_point), axis=0)
        transpose_matrix = np.vstack((np.transpose(bottom_center_point),np.ones((1,1))))
        
        homogeneous_coordinates = np.matmul(sensor_calibration_dict['camera_to_ground'], transpose_matrix)
        ground_coordinates = homogeneous_coordinates / homogeneous_coordinates[-1].reshape(1, -1)

        transpose_ground_coordinates = ground_coordinates.T
        g_x1y1 = transpose_ground_coordinates[0][:2]
        # g_x2y2 = transpose_ground_coordinates[1][:2]
        # g_xcyc = transpose_ground_coordinates[2][:2]

        # ground_coordinate_list.append([list(g_x1y1), list(g_x2y2), list(g_xcyc)])
        ground_coordinate_list.append([list(g_x1y1)])

    # print(ground_coordinate_list)
    return ground_coordinate_list

'''

"\ndef homography2(list_of_pred_boxes):\n    ground_coordinate_list = []\n    for result in list_of_pred_boxes:\n        bbox = list(result[1:5])\n    \n        # x1y1 = np.array(bbox[:2]).reshape(1, -1)\n        # x2y2 = np.array(bbox[2:]).reshape(1, -1)\n        bottom_center_point = np.array(list(((bbox[2] + bbox[0]) / 2, bbox[3]))).reshape(1, -1) \n\n        # image_coordinates = np.concatenate((x1y1,x2y2,bottom_center_point), axis=0)\n        # image_coordinates = np.concatenate((bottom_center_point), axis=0)\n        transpose_matrix = np.vstack((np.transpose(bottom_center_point),np.ones((1,1))))\n        \n        homogeneous_coordinates = np.matmul(sensor_calibration_dict['camera_to_ground'], transpose_matrix)\n        ground_coordinates = homogeneous_coordinates / homogeneous_coordinates[-1].reshape(1, -1)\n\n        transpose_ground_coordinates = ground_coordinates.T\n        g_x1y1 = transpose_ground_coordinates[0][:2]\n        # g_x2y2 = transpose_ground_coordinates[1][:2]\

In [3177]:
def homography(points_on_image, sensor_calibration_dict):

    points = np.array(points_on_image).reshape(1, -1)

    transpose_matrix = np.vstack((np.transpose(points),np.ones((1,1))))
    
    homogeneous_coordinates = np.matmul(sensor_calibration_dict['camera_to_ground'], transpose_matrix)
    ground_coordinates = homogeneous_coordinates / homogeneous_coordinates[-1].reshape(1, -1)

    transpose_ground_coordinates = ground_coordinates.T
    g_x1y1 = transpose_ground_coordinates[0][:2]

    return g_x1y1

__Visualization: Camera Points on Ground Plane__

In [3178]:
def camera_plotting(image_on_ground, my_plot):
        x_plotting_list = []
        y_plotting_list = []

        for xy in image_on_ground:
                x_coords = [xy[1][0]]
                y_coords = [xy[1][1]]

                x_plotting_list.append(x_coords)
                y_plotting_list.append(y_coords)
        
        colors = ['blue', 'green', 'orange', 'black', 'purple', 'maroon']

        for i, (x_co, y_co) in enumerate(zip(x_plotting_list, y_plotting_list)):
                my_plot.scatter(y_co, x_co, color=colors[i], label= 'camera', marker='o')

        my_plot.set_xlim(-30,30)
        my_plot.set_ylim(0,100)
        my_plot.set_xlabel('Y-axis')
        my_plot.set_ylabel('X-axis')
        my_plot.set_title('Plot of Points')
        my_plot.invert_xaxis()
        
        return my_plot

__Visualization: Radar points on Ground Plane__

In [3179]:
def radar_plotting(dict, my_plot):
    clusters = dict['clusters']
    noise_points = dict['noise']

    x_lowest = []
    y_lowest = []

    x_noise = []
    y_noise = []

    for detection in clusters:
        if len(detection) != 0:
            lowest_point = detection[1]
            x_lp = lowest_point[0]
            y_lp = lowest_point[1]
            x_lowest.append(x_lp)
            y_lowest.append(y_lp)
    
    for noise in noise_points:
        if len(noise) != 0:
            lowest_point = noise[0]
            x_n = lowest_point[0]
            y_n = lowest_point[1]
            x_noise.append(x_n)
            y_noise.append(y_n)


    my_plot.scatter(y_lowest, x_lowest, color='red', label='lowest point', marker="X")
    my_plot.scatter(y_noise, x_noise, color='grey', label='Noise', marker=".")

    if not my_plot.get_legend():
        my_plot.legend()

    return my_plot

__Expand Bounding Box Dimension__

In [3180]:
def expand_bbox(box, scale=1.1):
    # Calculate the width and height of the original box
    width = box[2] - box[0]     # x2 - x1
    height = box[3] - box[1]    # y2 - y1

    # Calculate the center of the original box
    center_x = box[0] + (width/2)
    center_y = box[1] + (height/2)

    # Calculate the increase in width and height
    new_width = width * scale
    new_height = height * scale

    # Calculate the new coordinates
    new_x1 = 0 if (center_x - new_width / 2) < 0 else (center_x - new_width / 2)
    new_y1 = 0 if (center_y - new_height / 2) < 0 else (center_y - new_height / 2)
    new_x2 = center_x + new_width / 2
    new_y2 = center_y + new_height / 2
    
    return list([new_x1, new_y1, new_x2, new_y2])


__Euclidean Distance__

In [3181]:
def get_euclidean_distance(clusters, images):
    d = np.sqrt(((clusters[0] - images[0])**2) + ((clusters[1] - images[1])**2))
    return d


__Case Filtering__

In [3182]:
def get_association_matrix(list_of_pred_boxes, cluster_on_image, datatype='clusters', association_list=None):

    clusters = list(cluster_on_image['clusters']) if datatype == 'clusters' else list(cluster_on_image['noise'])
    pred_boxes = list(list_of_pred_boxes)

    if len(clusters) > 0 and len(pred_boxes) > 0:
        matrix = np.zeros((len(clusters), len(pred_boxes)))

        for pred_idx, prediction in enumerate(pred_boxes):
            old_bbox = prediction[1:5]  
            bbox = expand_bbox(old_bbox, scale=1.2)

            for cluster_idx, cluster in enumerate(clusters):
                cluster_centroid = cluster[0]
                
                if datatype == 'clusters':  
                    if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                        matrix[cluster_idx, pred_idx] = 1
                    else: 
                        matrix[cluster_idx, pred_idx] = 0

                elif datatype == 'noise':
                    if pred_idx in association_list['non_associated_bbox']:
                        if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                            matrix[cluster_idx, pred_idx] = 1
                        else: 
                            matrix[cluster_idx, pred_idx] = 0

        return matrix 
    

In [3183]:
'''
def get_association_matrix(list_of_pred_boxes, cluster_on_image, datatype='clusters', association_list=None):

    if datatype == 'clusters':
        clusters = list(cluster_on_image['clusters']) 
        pred_boxes = list(list_of_pred_boxes)

        if len(clusters) > 0 and len(pred_boxes)>0:
            matrix = np.zeros((len(clusters), len(pred_boxes))) 
            for pred_idx, prediction in enumerate(pred_boxes):
                
                old_bbox = prediction[1:5]  
                bbox = expand_bbox(old_bbox, scale=1.2)

                for cluster_idx, cluster in enumerate(clusters):
                    cluster_centroid = cluster[0]
                    
                    if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                        matrix[cluster_idx, pred_idx] = 1

                    else: 
                        matrix[cluster_idx, pred_idx] = 0

            return matrix 


    elif datatype == 'noise':
        clusters = list(cluster_on_image['noise'])
        pred_boxes = list(list_of_pred_boxes)
        # pred_boxes = [list(box) for idx, box in enumerate(list_of_pred_boxes) if idx in association_list['non_associated_bbox']]

        if len(clusters) > 0 and len(pred_boxes)>0:
            matrix = np.zeros((len(clusters), len(pred_boxes))) 
            for pred_idx, prediction in enumerate(pred_boxes):
                
                old_bbox = prediction[1:5]  
                bbox = expand_bbox(old_bbox, scale=1.2)

                for cluster_idx, cluster in enumerate(clusters):
                    cluster_centroid = cluster[0]

                    if pred_idx  not in association_list['non_associated_bbox']:
                        if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                            matrix[cluster_idx, pred_idx] = 1

                    else: 
                        matrix[cluster_idx, pred_idx] = 0


        return matrix 

    
    else:
        print('KeyError')
    '''


"\ndef get_association_matrix(list_of_pred_boxes, cluster_on_image, datatype='clusters', association_list=None):\n\n    if datatype == 'clusters':\n        clusters = list(cluster_on_image['clusters']) \n        pred_boxes = list(list_of_pred_boxes)\n\n        if len(clusters) > 0 and len(pred_boxes)>0:\n            matrix = np.zeros((len(clusters), len(pred_boxes))) \n            for pred_idx, prediction in enumerate(pred_boxes):\n                \n                old_bbox = prediction[1:5]  \n                bbox = expand_bbox(old_bbox, scale=1.2)\n\n                for cluster_idx, cluster in enumerate(clusters):\n                    cluster_centroid = cluster[0]\n                    \n                    if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:\n                        matrix[cluster_idx, pred_idx] = 1\n\n                    else: \n                        matrix[cluster_idx, pred_idx] = 0\n\n            return matrix \n\n\n    elif da

In [3184]:
'''
def get_association_matrix(list_of_pred_boxes, cluster_on_image):

    clusters = list(cluster_on_image['clusters'])
    # noise_points = list(cluster_on_image['noise'])
    pred_boxes = list(list_of_pred_boxes)

    if len(clusters) > 0 and len(pred_boxes)>0:
        matrix = np.zeros((len(clusters), len(pred_boxes))) 
        for pred_idx, prediction in enumerate(pred_boxes):
            
            old_bbox = prediction[1:5]  
            bbox = expand_bbox(old_bbox, scale=1.2)

            for cluster_idx, cluster in enumerate(clusters):
                cluster_centroid = cluster[0]
                
                if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                    matrix[cluster_idx, pred_idx] = 1

                else: 
                    matrix[cluster_idx, pred_idx] = 0

        return matrix 
'''      


"\ndef get_association_matrix(list_of_pred_boxes, cluster_on_image):\n\n    clusters = list(cluster_on_image['clusters'])\n    # noise_points = list(cluster_on_image['noise'])\n    pred_boxes = list(list_of_pred_boxes)\n\n    if len(clusters) > 0 and len(pred_boxes)>0:\n        matrix = np.zeros((len(clusters), len(pred_boxes))) \n        for pred_idx, prediction in enumerate(pred_boxes):\n            \n            old_bbox = prediction[1:5]  \n            bbox = expand_bbox(old_bbox, scale=1.2)\n\n            for cluster_idx, cluster in enumerate(clusters):\n                cluster_centroid = cluster[0]\n                \n                if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:\n                    matrix[cluster_idx, pred_idx] = 1\n\n                else: \n                    matrix[cluster_idx, pred_idx] = 0\n\n        return matrix \n"

In [3185]:
'''
def get_noise_association_matrix(association_list, list_of_pred_boxes, cluster_on_image):

    # clusters = list(cluster_on_image['clusters'])
    noise_points = list(cluster_on_image['noise'])

    if not association_list['non_associated_bbox']:
        pred_boxes = list(list_of_pred_boxes)
    else:
        pred_boxes = [list(box) for idx, box in enumerate(list_of_pred_boxes) if idx in association_list['non_associated_bbox']]

    pprint(pred_boxes)


    if len(noise_points) > 0 and len(pred_boxes)>0:
        matrix = np.zeros((len(noise_points), len(pred_boxes))) 
        for pred_idx, prediction in enumerate(pred_boxes):
            
            old_bbox = prediction[1:5]  
            bbox = expand_bbox(old_bbox, scale=1.2)

            for noise_idx, noise in enumerate(noise_points):
                noise_centroid = noise[0]
                
                if bbox[0] < noise_centroid[0] < bbox[2] and bbox[1] < noise_centroid[1] < bbox[3]:
                    matrix[noise_idx, pred_idx] = 1

                else: 
                    matrix[noise_idx, pred_idx] = 0

        return matrix 

'''

"\ndef get_noise_association_matrix(association_list, list_of_pred_boxes, cluster_on_image):\n\n    # clusters = list(cluster_on_image['clusters'])\n    noise_points = list(cluster_on_image['noise'])\n\n    if not association_list['non_associated_bbox']:\n        pred_boxes = list(list_of_pred_boxes)\n    else:\n        pred_boxes = [list(box) for idx, box in enumerate(list_of_pred_boxes) if idx in association_list['non_associated_bbox']]\n\n    pprint(pred_boxes)\n\n\n    if len(noise_points) > 0 and len(pred_boxes)>0:\n        matrix = np.zeros((len(noise_points), len(pred_boxes))) \n        for pred_idx, prediction in enumerate(pred_boxes):\n            \n            old_bbox = prediction[1:5]  \n            bbox = expand_bbox(old_bbox, scale=1.2)\n\n            for noise_idx, noise in enumerate(noise_points):\n                noise_centroid = noise[0]\n                \n                if bbox[0] < noise_centroid[0] < bbox[2] and bbox[1] < noise_centroid[1] < bbox[3]:\n            

In [3186]:
def get_filtered_cases(matrix, datatype='clusters', association_list=None):
    """
    Checks and assigns different cases of radar-image data for spatial association 

    Examples: 
    >>> import pprint
    >>> matrix = np.array([
    ...     [1, 0, 1, 0, 0, 0],
    ...     [0, 1, 1, 0, 0, 0],
    ...     [1, 1, 0, 0, 0, 0],
    ...     [0, 0, 0, 1, 0, 0],
    ...     [0, 0, 0, 0, 1, 0],
    ...     [0, 0, 0, 0, 1, 0],
    ...     [0, 0, 0, 0, 0, 0]
    ... ])
    >>> pprint.pprint(get_associations(matrix))
    {'many_radar_to_many_image': {'cols': [0, 1, 2], 'rows': [0, 1, 2]},
     'many_radar_to_one_image': {'cols': [4], 'rows': [(array([4, 5]),)]},
     'one_radar_to_many_image': {'cols': [], 'rows': []},
     'one_radar_to_one_image': {'cols': [3], 'rows': [3]}}
    """ 
    associations = {
        "many_cluster_to_many_bbox" : {"clusters": [], "bbox": []}, 
        "many_cluster_to_one_bbox"  : {"clusters": [], "bbox": []},
        "one_cluster_to_many_bbox"  : {"clusters": [], "bbox": []}, 
        "one_cluster_to_one_bbox"   : {"assigned": []},
        "unassigned_bbox" : {"bbox": []}
    }

    # MANY TO MANY CHECKS
    # -------------------
    rows_with_multiple_truths = np.where(np.sum(matrix, axis=1) > 1)[0] 
    columns_with_multiple_truths = np.where(np.sum(matrix, axis=0) > 1)[0] 
    # many_too_many = list(set(rows_with_multiple_truths) & set(columns_with_multiple_truths))
    # many_radar_to_many_image = [many_too_many, many_too_many]
    many_too_many_rows = set() 
    many_too_many_cols = set() 
    for r_id in range(matrix.shape[0]):
        for c_id in range(matrix.shape[1]):
            if r_id in rows_with_multiple_truths and c_id in columns_with_multiple_truths: 
                if matrix[r_id, c_id] == 1:
                     many_too_many_rows.add(r_id)
                     many_too_many_cols.add(c_id)

    associations['many_cluster_to_many_bbox']["clusters"] = list(many_too_many_rows)
    associations['many_cluster_to_many_bbox']["bbox"] = list(many_too_many_cols)

    # MANY TO ONE CHECKS 
    # -------------------
    many_to_one = [] 
    for c in range(matrix.shape[1]): 
        if c in columns_with_multiple_truths and c not in many_too_many_cols:
            associated_rows = np.where(matrix[:, c] > 0)[0]
            associations['many_cluster_to_one_bbox']["clusters"].append(associated_rows.tolist())
            associations['many_cluster_to_one_bbox']["bbox"].append([c])            

    # ONE TO MANY CHECKS
    # ------------------
    one_to_many = [] 
    for r in range(matrix.shape[0]): 
        if r in rows_with_multiple_truths and r not in many_too_many_rows:
            associated_cols = np.where(matrix[r] > 0)[0]
            associations['one_cluster_to_many_bbox']["clusters"].append([r])
            associations['one_cluster_to_many_bbox']["bbox"].append(associated_cols.tolist())

    # ONE TO ONE CHECKS
    # ------------------
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i,j] == 1:
                row_sum = sum(matrix[i,:])
                col_sum = sum(matrix[:,j]) 

                if row_sum == 1 and col_sum == 1:
                    associations['one_cluster_to_one_bbox']['assigned'].append([i, j]) 

    if datatype == 'clusters':
        associations["unassigned_bbox"]["bbox"].extend(list(np.where(np.sum(matrix, axis=0) == 0)[0]))
    elif datatype == 'noise':
        associations["unassigned_bbox"]["bbox"].extend(box for box in list(np.where(np.sum(matrix, axis=0) == 0)[0]) if box in association_list['non_associated_bbox']) 

    return associations

__Association: One to One @Image Plane__

In [3187]:
def get_one_to_one_association(filtered_cases, association_list):
    association_list["associated"].extend(filtered_cases['one_cluster_to_one_bbox']['assigned']) 
    association_list["non_associated_bbox"].extend(filtered_cases["unassigned_bbox"]["bbox"]) 
    return association_list

In [3188]:
'''
def get_one_one_association(list_of_pred_boxes, cluster_on_image):
    
    clusters = list(cluster_on_image['clusters'])
    noise_points = list(cluster_on_image['noise'])
    pred_boxes = list(list_of_pred_boxes)

    association = {'associated': [], 'non_associated':{'YOLO':[], 'Radar':[]}}

    if len(clusters) > 0 and len(pred_boxes)>0:
        matrix = np.zeros((len(clusters), len(pred_boxes))) 
        for pred_idx, prediction in enumerate(pred_boxes):
            bbox = prediction[1:5]  
            for cluster_idx, cluster in enumerate(clusters):
                cluster_centroid = cluster[0]
                
                if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:
                    matrix[cluster_idx, pred_idx] = 1

                else: 
                    matrix[cluster_idx, pred_idx] = 0
        
        pprint(matrix)

        special_points = []
        for i in range(len(matrix)):
            for j in range(len(matrix[0])):
                if matrix[i,j] == 1:
                    row_sum = sum(matrix[i,:])
                    col_sum = sum(matrix[:,j]) 

                    if row_sum == 1 and col_sum == 1:
                        special_points.append((i, j))
            
        pprint(special_points)

        for item in special_points:
            association['associated'].append([pred_boxes[item[1]],clusters[item[0]]])

        for i in range(matrix.shape[0]):
            if not any(i == item[0] for item in special_points):
                association["non_associated"]["Radar"].append(clusters[i])
            
        for j in range(matrix.shape[1]):
            if not any(j == item[1] for item in special_points):
                association["non_associated"]["YOLO"].append(pred_boxes[j])
            
                    
    return association
    '''

'\ndef get_one_one_association(list_of_pred_boxes, cluster_on_image):\n    \n    clusters = list(cluster_on_image[\'clusters\'])\n    noise_points = list(cluster_on_image[\'noise\'])\n    pred_boxes = list(list_of_pred_boxes)\n\n    association = {\'associated\': [], \'non_associated\':{\'YOLO\':[], \'Radar\':[]}}\n\n    if len(clusters) > 0 and len(pred_boxes)>0:\n        matrix = np.zeros((len(clusters), len(pred_boxes))) \n        for pred_idx, prediction in enumerate(pred_boxes):\n            bbox = prediction[1:5]  \n            for cluster_idx, cluster in enumerate(clusters):\n                cluster_centroid = cluster[0]\n                \n                if bbox[0] < cluster_centroid[0] < bbox[2] and bbox[1] < cluster_centroid[1] < bbox[3]:\n                    matrix[cluster_idx, pred_idx] = 1\n\n                else: \n                    matrix[cluster_idx, pred_idx] = 0\n        \n        pprint(matrix)\n\n        special_points = []\n        for i in range(len(matrix)):\n 

__Association: One to Many @Ground Plane__

In [3189]:
def get_one_to_many_association(filtered_cases, clusters_on_ground, image_on_ground, datatype='clusters', association_list=None):

    if datatype == 'clusters':
        lower_idx = 1
    elif datatype == 'noise':
        lower_idx = 0
    else:
        print('KeyError')

    list_of_centroid_indices = filtered_cases['one_cluster_to_many_bbox']['clusters']
    list_of_box_indices = filtered_cases['one_cluster_to_many_bbox']['bbox']

    for centroid, bboxes in zip(list_of_centroid_indices, list_of_box_indices):
        centroid = centroid[0]

        euclidean_distances = []
        for box in bboxes:
            ec_distance = get_euclidean_distance(clusters_on_ground[datatype][centroid][lower_idx], image_on_ground[box][1])
            euclidean_distances.append(ec_distance)
        
        min_bbox_idx = [idx for idx, value in enumerate(euclidean_distances) if value == min(euclidean_distances)]
        min_bbox_idx = min_bbox_idx[0]

        association_list['associated'].append([centroid, bboxes[min_bbox_idx]])
        association_list['non_associated_bbox'].extend([bbox for bbox in bboxes if bbox != bboxes[min_bbox_idx]])

    return association_list

__Association: Many to One__

In [3190]:
def nearest_point_finder(points_list, processed_radar_points_to_ground):
    x_list = []
    for point in points_list:
        x_p = processed_radar_points_to_ground['clusters'][point][1][0]
        x_list.append(x_p)
    
    return x_list.index(min(x_list))

In [3191]:
def get_many_to_one_association(filtered_cases, association_list, clusters_on_ground, image_on_ground, datatype='clusters'):

    clusters = filtered_cases['many_cluster_to_one_bbox']['clusters']
    pre_boxes = filtered_cases['many_cluster_to_one_bbox']['bbox']

  
    for cluster, box in zip(clusters, pre_boxes):
    # if count_in_clusters == 1 and count_in_boxes == 1:
        box_data = box[0]
        clusters_data = cluster
        count = len(clusters_data)
        matrix = np.zeros((count, count))

        for index_p1, point_1 in enumerate(clusters_data):
            for index_p2, point_2 in enumerate(clusters_data):
                point_1_data = clusters_on_ground['clusters'][point_1] if datatype == 'clusters' else clusters_on_ground['noise'][point_1]
                point_2_data = clusters_on_ground['clusters'][point_2] if datatype == 'clusters' else clusters_on_ground['noise'][point_2]

                velocity_p1 = point_1_data[2][0] if datatype == 'clusters' else point_1_data[1][0]
                velocity_p2 = point_2_data[2][0] if datatype == 'clusters' else point_2_data[1][0]

                x_p1 = point_1_data[0][0]
                x_p2 = point_2_data[0][0]
                
                if abs(velocity_p1 - velocity_p2) < 0.75 and abs(x_p1 - x_p2) < 2:
                    matrix[index_p1][index_p2] = 1

        candidates = []
        for row in matrix:
            columns_with_ones = []
            for col_index, value in enumerate(row):
                if value == 1:
                    columns_with_ones.append(col_index)
        
            candidates.append(columns_with_ones)

        new_candidates = candidates
        global_merged_any = False
        while True:
            #print(new_candidates)
            check_list = new_candidates
            for i in range(len(new_candidates)):
                local_merged_any = False
                set1 = set(new_candidates[i])
                for j in range(i + 1, len(new_candidates)):
                    set2 = set(new_candidates[j])

                    if set1 & set2:  # Check if there's any common element
                        new_element = list(set1 | set2)
                        remaining_points = [point for point in new_candidates if point not in [new_candidates[i], new_candidates[j]]]
                        new_candidates = remaining_points
                        new_candidates.insert(0, new_element) # New candidate is not emptying --no break condition 

                        local_merged_any = True 
                        global_merged_any = True
                        break
                if local_merged_any:
                    break

            # Exit condition for the while loop
            if len(new_candidates) == len(check_list):
                break
                    
        if not global_merged_any:  # If no merges occurred in this iteration, terminate the loop
            distance_data = []
            for idx, centroid in enumerate(clusters_data):
                nearest_data = clusters_on_ground['clusters'][centroid][1] if datatype == 'clusters' else clusters_on_ground['noise'][centroid][0]
                box_bottom_center = image_on_ground[box_data][1]
                ec_distance = get_euclidean_distance(nearest_data[0:2], box_bottom_center)
                distance_data.append(ec_distance)
            
            index_to_look_for_in_clusters_data = distance_data.index(min(distance_data))
            index_of_chosen_centroid = clusters_data[index_to_look_for_in_clusters_data]

            if [index_of_chosen_centroid, box_data] not in association_list['associated']:
                association_list['associated'].append([index_of_chosen_centroid, box_data])
            print('not merged case')

        new_updated_candidate = []
        for box_list in new_candidates:
            updated = []
            for box in box_list:
                updated.append(clusters_data[box]) 
            new_updated_candidate.append(updated)
            
        if len(new_updated_candidate) > 1 and global_merged_any:

            #This section removes the items which are not clustered
            for pair in new_updated_candidate[:]:
                if len(pair) < 2:
                    new_updated_candidate.remove(pair)    

            distance_comparison = []
            for item in new_updated_candidate:
                nearest_point = nearest_point_finder(item, clusters_on_ground)
                nearest_point_data = clusters_on_ground['clusters'][item[nearest_point]][1] if datatype == 'clusters' else clusters_on_ground['noise'][item[nearest_point]][0]
                box_bottom_center = image_on_ground[box_data][1]
                e_c_distance = get_euclidean_distance(nearest_point_data[0:2], box_bottom_center)
                distance_comparison.append(e_c_distance)

            final_cluster_to_associate= distance_comparison.index(min(distance_comparison))
            index_of_chosen_centroid = new_updated_candidate[final_cluster_to_associate]

            #print(final_candidate, box_data)
            association_list['associated'].append([index_of_chosen_centroid, box_data])

        elif len(new_updated_candidate) == 1:
            association_list['associated'].append([new_updated_candidate[0], box_data])
        
    return association_list

__Many to Many__

In [3192]:
def get_many_to_many_association(filtered_cases, association_list, clusters_on_ground, image_on_ground, datatype='clusters'):

    clusters = filtered_cases['many_cluster_to_one_bbox']['clusters']
    pre_boxes = filtered_cases['many_cluster_to_one_bbox']['bbox']

    
    first_matrix = np.zeros((len(clusters), len(clusters)))

    for index_p1, point_1 in enumerate(clusters):
            for index_p2, point_2 in enumerate(clusters):
                point_1_data = clusters_on_ground['clusters'][point_1] if datatype == 'clusters' else clusters_on_ground['noise'][point_1]
                point_2_data = clusters_on_ground['clusters'][point_2] if datatype == 'clusters' else clusters_on_ground['noise'][point_2]

                velocity_p1 = point_1_data[2][0] if datatype == 'clusters' else point_1_data[1][0]
                velocity_p2 = point_2_data[2][0] if datatype == 'clusters' else point_2_data[1][0]
                
                if abs(velocity_p1 - velocity_p2) < 0.75:
                    first_matrix[index_p1][index_p2] = 1
    
    # print(first_matrix)

    candidates = []
    for row in first_matrix:
        columns_with_ones = []
        for col_index, value in enumerate(row):
            if value == 1:
                columns_with_ones.append(col_index)
    
        candidates.append(columns_with_ones)

    #########################################################################################################################
    new_candidates = candidates
    global_merged_any = False
    while True:
        #print(new_candidates)
        check_list = new_candidates
        for i in range(len(new_candidates)):
            local_merged_any = False
            set1 = set(new_candidates[i])
            for j in range(i + 1, len(new_candidates)):
                set2 = set(new_candidates[j])

                if set1 & set2:  # Check if there's any common element
                    new_element = list(set1 | set2)
                    remaining_points = [point for point in new_candidates if point not in [new_candidates[i], new_candidates[j]]]
                    new_candidates = remaining_points
                    new_candidates.insert(0, new_element) # New candidate is not emptying --no break condition 

                    local_merged_any = True 
                    global_merged_any = True
                    break
            if local_merged_any:
                break

        # Exit condition for the while loop
        if len(new_candidates) == len(check_list):
            break

    ################################################################################################################
    if not global_merged_any:  # If no merges occurred in this iteration, terminate the loop

        second_matrix = np.zeros((len(clusters), len(pre_boxes)))

        for cluster in clusters:
            for box in pre_boxes:
                cluster_nearest_point_data = clusters_on_ground['clusters'][cluster][1]  if datatype == 'clusters' else clusters_on_ground['noise'][cluster][0]
                box_bottom_center_data = image_on_ground[box][1]
                e_c_distance = get_euclidean_distance(cluster_nearest_point_data[0:2], box_bottom_center_data)
                second_matrix[cluster][box] = e_c_distance
        
        n_rows, n_cols = second_matrix.shape
        combined_indices = []

        for col_idx in range(n_cols):
            # Get the column and apply the threshold condition
            column = second_matrix.shape[:, col_idx]
            valid_indices = np.where(column < 2)[0]
            
            if valid_indices.size > 0:
                # Find the index of the minimum value among valid indices
                min_value_index = valid_indices[np.argmin(column[valid_indices])]
                combined_indices.append((min_value_index, col_idx))
        
        # print(combined_indices)             #list of associations where first element is cluster index and second is box index 
        for item in combined_indices:
            new_item = clusters[item[0]], pre_boxes[item[1]]
        association_list['associated'].extend(new_item)

    #####################################################################################################################

    #getting the actual indexes
    new_updated_candidate = []
    for box_list in new_candidates:
        updated = []
        for box in box_list:
            updated.append(clusters[box]) 
        new_updated_candidate.append(updated)

    if global_merged_any:
        number_of_rows = len(new_candidates)
        len_of_columns = len(pre_boxes)

        third_matrix = np.zeros((number_of_rows, len_of_columns))

        for merged in new_candidates:
            for box in pre_boxes:
                if len(merged) > 1:
                    nearest_point_index_in_merged = nearest_point_finder(merged, image_on_ground)
                    nearest_point = merged[nearest_point_index_in_merged]
                else:
                    nearest_point = merged[0]
                
                nearest_point_data = clusters_on_ground['cluster'][nearest_point][1] if datatype == 'clusters' else clusters_on_ground['noise'][nearest_point][0]
                box_bottom_center = image_on_ground[box][1]
                e_c_distance = get_euclidean_distance(nearest_point_data[0:2], box_bottom_center)
                third_matrix[merged][box] = e_c_distance
        

        n_rows, n_cols = third_matrix.shape
        combined_indices = []

        for col_idx in range(n_cols):
            # Get the column and apply the threshold condition
            column = third_matrix.shape[:, col_idx]
            valid_indices = np.where(column < 2)[0]
            
            if valid_indices.size > 0:
                # Find the index of the minimum value among valid indices
                min_value_index = valid_indices[np.argmin(column[valid_indices])]
                combined_indices.append((min_value_index, col_idx))
        
        # print(combined_indices)             #list of associations where first element is cluster index and second is box index 

        for item in combined_indices:
            new_item = new_candidates[item[0]], pre_boxes[item[1]]
        association_list['associated'].extend(new_item)
    
    for boxes in pre_boxes:
        if boxes not in list(item[1] for item in association_list['associated']):
            association_list['non_associated'].append(boxes)

    return association_list

__Jayesh__

In [3193]:
'''def find_and_group_similar_velocities(radar_on_camera, velocities, threshold=0.5, datatype='clusters'):
        grouped_points = []
        used_indices = set()
        if datatype == 'noise': 
            points = radar_on_camera['noise']
            v_index = 1 # Due to different structure, the position of velocity will be different
        else: 
            points = radar_on_camera['clusters']
            v_index = 2

        for i in velocities:
            if i not in used_indices:
                current_group = [i]
                used_indices.add(i)
                for j in velocities[i+1:]:
                    if abs(points[i][v_index][0] - points[j][v_index][0]) <= threshold:
                        current_group.append(j)
                        used_indices.add(j)
                grouped_points.append(current_group)
        
        return grouped_points'''

"def find_and_group_similar_velocities(radar_on_camera, velocities, threshold=0.5, datatype='clusters'):\n        grouped_points = []\n        used_indices = set()\n        if datatype == 'noise': \n            points = radar_on_camera['noise']\n            v_index = 1 # Due to different structure, the position of velocity will be different\n        else: \n            points = radar_on_camera['clusters']\n            v_index = 2\n\n        for i in velocities:\n            if i not in used_indices:\n                current_group = [i]\n                used_indices.add(i)\n                for j in velocities[i+1:]:\n                    if abs(points[i][v_index][0] - points[j][v_index][0]) <= threshold:\n                        current_group.append(j)\n                        used_indices.add(j)\n                grouped_points.append(current_group)\n        \n        return grouped_points"

In [3194]:
'''def merge_clusters(clusters_on_radar, clusters, datatype='clusters'):
        avg_centroid = np.empty((1,3))
        avg_lowest_point = np.empty((1,3))
        avg_velocity = 0
        if datatype=='noise': 
            points = clusters_on_radar['noise']
            v_index = 1
        else: 
            points = clusters_on_radar['clusters']
            v_index = 2
        
        for cluster in clusters:
            # print(f"Cluster: {cluster}")
            # print(f"Points: {points}")
            avg_centroid += np.array(points[cluster][0])
            avg_lowest_point += np.array(points[cluster][1])
            avg_velocity += points[cluster][v_index][0] 
        
        avg_centroid /= len(clusters)
        avg_lowest_point /= len(clusters)
        avg_velocity /= len(clusters)

        merged_cluster = [avg_centroid.tolist()[0], avg_lowest_point.tolist()[0], [avg_velocity]]

        return merged_cluster
'''


'def merge_clusters(clusters_on_radar, clusters, datatype=\'clusters\'):\n        avg_centroid = np.empty((1,3))\n        avg_lowest_point = np.empty((1,3))\n        avg_velocity = 0\n        if datatype==\'noise\': \n            points = clusters_on_radar[\'noise\']\n            v_index = 1\n        else: \n            points = clusters_on_radar[\'clusters\']\n            v_index = 2\n        \n        for cluster in clusters:\n            # print(f"Cluster: {cluster}")\n            # print(f"Points: {points}")\n            avg_centroid += np.array(points[cluster][0])\n            avg_lowest_point += np.array(points[cluster][1])\n            avg_velocity += points[cluster][v_index][0] \n        \n        avg_centroid /= len(clusters)\n        avg_lowest_point /= len(clusters)\n        avg_velocity /= len(clusters)\n\n        merged_cluster = [avg_centroid.tolist()[0], avg_lowest_point.tolist()[0], [avg_velocity]]\n\n        return merged_cluster\n'

In [3195]:
'''
def get_many_to_many(filtered_cases, clusters_on_image, velocity_threshold=0.75, datatype='clusters')
    if datatype == 'clusters':
        merged_clusters_indexes = find_and_group_similar_velocities(clusters_on_image, filtered_cases['many_cluster_to_many_bbox']["clusters"], threshold=0.5, datatype='clusters')
        merged_clusters_indexes_copy = merged_clusters_indexes

        merged_clusters = [merge_clusters(clusters) for clusters in merged_clusters_indexes if len(clusters)>1]
'''
        

'\ndef get_many_to_many(filtered_cases, clusters_on_image, velocity_threshold=0.75, datatype=\'clusters\')\n    if datatype == \'clusters\':\n        merged_clusters_indexes = find_and_group_similar_velocities(clusters_on_image, filtered_cases[\'many_cluster_to_many_bbox\']["clusters"], threshold=0.5, datatype=\'clusters\')\n        merged_clusters_indexes_copy = merged_clusters_indexes\n\n        merged_clusters = [merge_clusters(clusters) for clusters in merged_clusters_indexes if len(clusters)>1]\n'

In [3196]:
'''
def get_noise_one_to_many_association(filtered_cases, noise_association_list, clusters_on_ground, image_on_ground):

    list_of_noise_indices = filtered_cases['one_cluster_to_many_bbox']['clusters']
    list_of_box_indices = filtered_cases['one_cluster_to_many_bbox']['bbox']

    for centroid, bboxes in zip(list_of_noise_indices, list_of_box_indices):
        centroid = centroid[0]

        euclidean_distances = []
        for box in bboxes:
            ec_distance = get_euclidean_distance(clusters_on_ground['noise'][centroid][0], image_on_ground[box][1])
            euclidean_distances.append(ec_distance)
        
        min_bbox_idx = [idx for idx, value in enumerate(euclidean_distances) if value == min(euclidean_distances)]
        min_bbox_idx = min_bbox_idx[0]

        noise_association_list['associated'].append([centroid, bboxes[min_bbox_idx]])
        noise_association_list['non_associated_bbox'].extend([bbox for bbox in bboxes if bbox != bboxes[min_bbox_idx]])

    return noise_association_list

    '''

"\ndef get_noise_one_to_many_association(filtered_cases, noise_association_list, clusters_on_ground, image_on_ground):\n\n    list_of_noise_indices = filtered_cases['one_cluster_to_many_bbox']['clusters']\n    list_of_box_indices = filtered_cases['one_cluster_to_many_bbox']['bbox']\n\n    for centroid, bboxes in zip(list_of_noise_indices, list_of_box_indices):\n        centroid = centroid[0]\n\n        euclidean_distances = []\n        for box in bboxes:\n            ec_distance = get_euclidean_distance(clusters_on_ground['noise'][centroid][0], image_on_ground[box][1])\n            euclidean_distances.append(ec_distance)\n        \n        min_bbox_idx = [idx for idx, value in enumerate(euclidean_distances) if value == min(euclidean_distances)]\n        min_bbox_idx = min_bbox_idx[0]\n\n        noise_association_list['associated'].append([centroid, bboxes[min_bbox_idx]])\n        noise_association_list['non_associated_bbox'].extend([bbox for bbox in bboxes if bbox != bboxes[min_bbox_id

__Make Association Matrix__

In [3197]:
'''
def get_association_matrix(image_points_on_ground, radar_points):
    clusters = radar_points['clusters']
    noise_points = radar_points['noise']
    
    cluster_nearest_points = []
    noise_nearest_points = []

    for detection in clusters: 
        if len(detection) != 0:
            lowest_point = detection[1]
            x_lp = lowest_point[0]
            y_lp = lowest_point[1]
            cluster_nearest_points.append([list(np.array([x_lp, y_lp]))]) 
    
    for noise in noise_points: 
        if len(noise) != 0:
            lowest_point = noise[0]
            x_n = lowest_point[0]
            y_n = lowest_point[1]
            noise_nearest_points.append([list(np.array([x_n, y_n]))])     
    

    total_list = cluster_nearest_points + noise_nearest_points

    association_matrix = np.zeros((len(total_list),len(image_points_on_ground)))

    for cluster_idx, cluster_point in enumerate(total_list):
        for img_idx, img_point in enumerate(image_points_on_ground):
            association_matrix[cluster_idx, img_idx] = get_euclidean_distance(cluster_point[0], img_point[0])

    
    return association_matrix   

'''

"\ndef get_association_matrix(image_points_on_ground, radar_points):\n    clusters = radar_points['clusters']\n    noise_points = radar_points['noise']\n    \n    cluster_nearest_points = []\n    noise_nearest_points = []\n\n    for detection in clusters: \n        if len(detection) != 0:\n            lowest_point = detection[1]\n            x_lp = lowest_point[0]\n            y_lp = lowest_point[1]\n            cluster_nearest_points.append([list(np.array([x_lp, y_lp]))]) \n    \n    for noise in noise_points: \n        if len(noise) != 0:\n            lowest_point = noise[0]\n            x_n = lowest_point[0]\n            y_n = lowest_point[1]\n            noise_nearest_points.append([list(np.array([x_n, y_n]))])     \n    \n\n    total_list = cluster_nearest_points + noise_nearest_points\n\n    association_matrix = np.zeros((len(total_list),len(image_points_on_ground)))\n\n    for cluster_idx, cluster_point in enumerate(total_list):\n        for img_idx, img_point in enumerate(image

__Greedy Object Association__

In [3198]:
'''
def assign_objects(_matrix):
    
    for i_dx in range(_matrix.shape[0]):
        _matrix[i_dx,][_matrix[i_dx,] != np.max(_matrix[i_dx,])] = 0

    for i_gx in range(_matrix.shape[1]):
        _matrix[:, i_gx][_matrix[:,i_gx] != np.max(_matrix[:,i_gx])] = 0 

    return _matrix 
'''


'\ndef assign_objects(_matrix):\n    \n    for i_dx in range(_matrix.shape[0]):\n        _matrix[i_dx,][_matrix[i_dx,] != np.max(_matrix[i_dx,])] = 0\n\n    for i_gx in range(_matrix.shape[1]):\n        _matrix[:, i_gx][_matrix[:,i_gx] != np.max(_matrix[:,i_gx])] = 0 \n\n    return _matrix \n'

__Update Dict: Image Plane to Ground Plane__

In [3199]:
'''
def update_dict_image_ground(dictionary_after_1t1, sensor_calibration_dict):
    _to_ground_dict = {'associated': [], 'non_associated': {'YOLO': [], 'Radar': []}}
    
    for result in dictionary_after_1t1['non_associated']['YOLO']:
        cls = result[0]
        bbox = list(result[1:5])
        bottom_center_point = list(((bbox[2] + bbox[0]) / 2, bbox[3]))
        image_point_on_ground = list(homography(bottom_center_point, sensor_calibration_dict))
        _to_ground_dict['non_associated']['YOLO'].append([[cls], image_point_on_ground])


    for cluster in dictionary_after_1t1['non_associated']['Radar']:
        centroid = cluster[0]
        lower = cluster[1]
        velocity = cluster[2]
        _centroid = list(homography(centroid,sensor_calibration_dict))
        _lower = list(homography(lower, sensor_calibration_dict))
        _to_ground_dict['non_associated']['Radar'].append([_centroid, _lower, velocity])


    for item in dictionary_after_1t1['associated']:
        cls = item[0][0]
        bbox_lower = item[0][1]
        radar_data = item[1]
        centroid = radar_data[0]
        velocity = radar_data[2]
        _centroid = list(homography(centroid, sensor_calibration_dict))
        _to_ground_dict['associated'].append([[cls], _centroid, velocity])

    return _to_ground_dict
'''

"\ndef update_dict_image_ground(dictionary_after_1t1, sensor_calibration_dict):\n    _to_ground_dict = {'associated': [], 'non_associated': {'YOLO': [], 'Radar': []}}\n    \n    for result in dictionary_after_1t1['non_associated']['YOLO']:\n        cls = result[0]\n        bbox = list(result[1:5])\n        bottom_center_point = list(((bbox[2] + bbox[0]) / 2, bbox[3]))\n        image_point_on_ground = list(homography(bottom_center_point, sensor_calibration_dict))\n        _to_ground_dict['non_associated']['YOLO'].append([[cls], image_point_on_ground])\n\n\n    for cluster in dictionary_after_1t1['non_associated']['Radar']:\n        centroid = cluster[0]\n        lower = cluster[1]\n        velocity = cluster[2]\n        _centroid = list(homography(centroid,sensor_calibration_dict))\n        _lower = list(homography(lower, sensor_calibration_dict))\n        _to_ground_dict['non_associated']['Radar'].append([_centroid, _lower, velocity])\n\n\n    for item in dictionary_after_1t1['associat

__Association Visualization @Image Plane__

In [3200]:
def get_drawings(colour_idx, image, box_data, centroid_data, datatype='clusters'):

    colors = [
        (255, 0, 0),    # Blue
        (0, 255, 0),    # Green
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 0, 0),    # Maroon
        (0, 128, 0),    # Dark Green
        (0, 0, 128),    # Navy
        (128, 128, 0)   # Olive
    ]

    # Calculate the Corners
    original_box = box_data[1:5]
    # pprint(original_box)
    corner_1 = tuple(map(int, [original_box[0], original_box[1]]))
    corner_2 = tuple(map(int, [original_box[2], original_box[3]]))            

    # Draw Original bounding box
    colour = colors[colour_idx]
    thickness_of_box = 3
    cv2.rectangle(image, corner_1, corner_2, colour, thickness_of_box)

    # expand bounding boxes and calculate the corners
    new_box = expand_bbox(box_data[1:5], scale=1.2) 
    corner_1 = tuple(map(int, [new_box[0], new_box[1]]))
    corner_2 = tuple(map(int, [new_box[2], new_box[3]]))
    

    # Draw a Expanded bounding box with dashed Rectangle 
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    thickness = 3
    dash_length = 10

    for i in range(corner_1[0], corner_2[0], dash_length * 2):
        for t in range(thickness):
            draw.line([(i, corner_1[1]+t), (i + dash_length, corner_1[1]+t)], fill=colour[::-1])
            draw.line([(i, corner_2[1]+t), (i + dash_length, corner_2[1]+t)], fill=colour[::-1])

    for i in range(corner_1[1], corner_2[1], dash_length * 2):
        for t in range(thickness):
            draw.line([(corner_1[0]+t, i), (corner_1[0]+t, i + dash_length)], fill=colour[::-1])
            draw.line([(corner_2[0]+t, i), (corner_2[0]+t, i + dash_length)], fill=colour[::-1])

    image = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    
    
    # Plot Centroid
    thickness = -1  # to fill the circle
    radius = 15
    
    if datatype == 'clusters':
        cluster_point = tuple(map(int, [centroid_data[0], centroid_data[1]]))
        image = cv2.circle(image, cluster_point, radius, colour, thickness)
    
    elif datatype == 'noise':     
        cluster_point_1 = tuple(map(int, [centroid_data[0]-12, centroid_data[1]-12]))
        cluster_point_2 = tuple(map(int, [centroid_data[0]+12, centroid_data[1]+12]))
        image = cv2.rectangle(image, cluster_point_1, cluster_point_2, colour, thickness=3)
    
    elif datatype == 'unassigned':
        pass

    return image

In [3201]:
def get_image_visualization(final_association_dict, list_of_pred_boxes, clusters_on_image, img):

    image = cv2.imread(img, cv2.IMREAD_COLOR)
    colour_idx = 0
    

    if final_association_dict['with_cluster']:
        for associated_point in final_association_dict['with_cluster']:

            # Get box and centroid data
            box_data = list_of_pred_boxes[associated_point[1]]

            if type(associated_point[0]) == int:
                point = associated_point[0]
                centroid_data = clusters_on_image['clusters'][point][0]
                
                # Get Annotations
                image = get_drawings(colour_idx, image, box_data, centroid_data, datatype='clusters')
                colour_idx += 1
            
            elif type(associated_point[0]) == list:
                for point in associated_point[0]:
                    centroid_data = clusters_on_image['clusters'][point][0]
                    
                    # Get Annotations
                    image = get_drawings(colour_idx, image, box_data, centroid_data, datatype='clusters')
                colour_idx += 1


    if final_association_dict['with_noise']:
        for associated_point in final_association_dict['with_noise']:

            # Get box and centroid data
            box_data = list_of_pred_boxes[associated_point[1]]

            if type(associated_point[0]) == int:
                point = associated_point[0]
                centroid_data = clusters_on_image['noise'][point][0]
                
                # Get Annotations
                image = get_drawings(colour_idx, image, box_data, centroid_data, datatype='noise')
                colour_idx += 1
            
            elif type(associated_point[0]) == list:
                for point in associated_point[0]:
                    centroid_data = clusters_on_image['noise'][point][0]
                    
                    # Get Annotations
                    image = get_drawings(colour_idx, image, box_data, centroid_data, datatype='noise')
                colour_idx += 1


    if final_association_dict['unassigned_bbox']:
        for non_associated_point in final_association_dict['unassigned_bbox']:
            # Get box and centroid data
            box_data = list_of_pred_boxes[non_associated_point]
            centroid_data = None

            # Get Annotations
            image = get_drawings(colour_idx, image, box_data, centroid_data, datatype='unassigned')
            colour_idx += 1

    return image

__Inference__

In [3202]:
%matplotlib qt
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
fig = plt.figure(figsize=(16, 9))

my_image = fig.add_subplot(gs[0])
my_plot = fig.add_subplot(gs[1])
plt.ion()


# total_box = []
# unassigned_case = []

# for scene_image, scene_pcd, calibration_file in zip(image_list, pcd_list, calibration_list):
sensor_calibration_dict = get_sensor_calibration_dict(calibration_file)

for img, pcd in zip(scene_image, scene_pcd):
    
    # YOLO prediction
    results = yolo_model.predict(img)
    list_of_pred_boxes = class_box_generator_for_pred(results)
    pprint(list_of_pred_boxes)
    
    # Calculate total no of boxes in all the images
    # total_box.append(len(list_of_pred_boxes))
    # print(total_box)

    # Cluster Formation
    db_scan = my_custom_dbscan(eps1=0.1, eps2=0.250, min_samples=2)
    clusters_on_radar = db_scan.process_pcd_files(pcd)
    pprint(clusters_on_radar)

    # Bbox point on the Ground Plane
    image_on_ground = []
    for result in list_of_pred_boxes:
        cls = result[0]
        bbox = list(result[1:5])
        bottom_center_point = list(((bbox[2] + bbox[0]) / 2, bbox[3]))
        image_point_on_ground = homography(bottom_center_point, sensor_calibration_dict)
        image_on_ground.append([[cls], list(image_point_on_ground)])
    # pprint(image_on_ground)

    # Radar Dictionaries on different Planes
    clusters_on_ground = radar_to_ground(clusters_on_radar, sensor_calibration_dict)
    # pprint(f"cluster on ground: {clusters_on_ground} ")

    clusters_on_image = radar_to_camera(clusters_on_radar, sensor_calibration_dict)
    # pprint(f"cluster on image: {clusters_on_image} ")

    
    # Fiter Cases with Association Matrix
    association_matrix = get_association_matrix(list_of_pred_boxes, clusters_on_image, datatype= 'clusters') 
    pprint(f" association_matrix: {association_matrix}")

    association_list = {"associated": [], "non_associated_bbox": []}

    if association_matrix is not None:
        filtered_cases = get_filtered_cases(association_matrix,  datatype='clusters')
        # pprint(f" filtered_cases: {filtered_cases}")

        # Association: One to One
        association_list = get_one_to_one_association(filtered_cases, association_list)
        # pprint(f" after one to one: {association_list}")

        # Association: One to Many
        association_list = get_one_to_many_association(filtered_cases, clusters_on_ground, image_on_ground, association_list=association_list, datatype= 'clusters')
        # pprint(f" after one to many: {association_list}")

        # Association: Many to One
        association_list = get_many_to_one_association(filtered_cases, association_list, clusters_on_ground, image_on_ground, datatype='clusters')
        # pprint(f" after many to one: {association_list}")

        # Association: Many to Many
        # association_list = get_many_to_many_association(filtered_cases, association_list, clusters_on_ground, image_on_ground, datatype='clusters')
        # pprint(f" after many to many: {association_list}")


        # Calculate Unassigned bbox without any clusters inside
        # unassigned_case.append(len(association_list["non_associated_bbox"][0]))
        # print(unassigned_case)

    noise_association_matrix = get_association_matrix(list_of_pred_boxes, clusters_on_image, association_list=association_list, datatype='noise')
    pprint(f"noise_association_matrix: {noise_association_matrix}")

    noise_association_list = {"associated": [], "non_associated_bbox": []}

    if noise_association_matrix is not None:
        noise_filtered_cases = get_filtered_cases(noise_association_matrix, datatype='noise', association_list=association_list)
        # pprint(f" noise filtered cases: {noise_filtered_cases}")

        # Association: One to One
        noise_association_list = get_one_to_one_association(noise_filtered_cases, noise_association_list)
        # pprint(f" after one to one: {noise_association_list}")

        # Association: One to Many
        noise_association_list = get_one_to_many_association(noise_filtered_cases, clusters_on_ground, image_on_ground, association_list=noise_association_list, datatype= 'noise')
        # pprint(f" after one to many: {noise_association_list}")

        # Association: Many to One    
        noise_association_list = get_many_to_one_association(noise_filtered_cases, noise_association_list, clusters_on_ground, image_on_ground, datatype='noise')
        # pprint(f" after Many to One: {noise_association_list}")


    # Final Association List
    final_association_dict = {'with_cluster': [], 'with_noise': [], 'unassigned_bbox': []}
    final_association_dict['with_cluster'].extend(association_list['associated'])
    final_association_dict['with_noise'].extend(noise_association_list['associated'])
    final_association_dict['unassigned_bbox'].extend(noise_association_list['non_associated_bbox'])
    
    # pprint(final_association_dict)

    
    # Visualization:
    my_image.clear()
    my_plot.clear()

    # Points on Ground Plane
    plot = camera_plotting(image_on_ground, my_plot)
    plot2 = radar_plotting(clusters_on_ground, plot)

    # Association Visualization on Image Plane
    image_visualize = get_image_visualization(final_association_dict, list_of_pred_boxes, clusters_on_image, img)
    # cv2.imshow('on image plane', image_visualize)
    image_visualize = cv2.cvtColor(image_visualize, cv2.COLOR_BGR2RGB)

    my_image.imshow(image_visualize)
    my_image.set_title('Image')

    fig.canvas.draw()
    # time.sleep(100)
    plt.pause(0.001)


# print(np.sum(total_box))
# print(np.sum(unassigned_case))


plt.ioff()



    


image 1/1 C:\Dk\Projects\Team Project\Dataset\INFRA-3DRC-Dataset\INFRA-3DRC_scene-15\camera_01\camera_01__data\camera_01__2023-06-02-21-28-09-314.png: 416x640 1 car, 75.0ms
Speed: 4.0ms preprocess, 75.0ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
array([[          3,      561.83,       521.6,      1225.7,      959.84,     0.95763]], dtype=float32)
{'clusters': [[]],
 'noise': [[[5.950912952423096, 1.220850944519043, -1.7535979747772217],
            [7.412753105163574]],
           [[6.699258327484131, 0.916589617729187, -1.7385363578796387],
            [8.009551048278809]],
           [[8.693258285522461, -0.6627280712127686, -2.348573684692383],
            [9.099832534790039]],
           [[74.41239929199219, -27.773658752441406, 0.39894238114356995],
            [-0.26798269152641296]],
           [[79.20195770263672, -11.261544227600098, 2.12113618850708],
            [-0.4293433725833893]],
           [[98.16716766357422, -11.847762107849121, -1.154921531

KeyboardInterrupt: 